In [1]:
import graphviz #graphiz is a graph visualization software
import sklearn 
#import DataFetcher class from file heart_data.py
import DataFetcher #importing the file DataFetcher.py from the folder data
data_fetcher = DataFetcher.DataFetcher("C:\\Users\\kruth\\OneDrive\\Desktop\\Cardiac_Events_ML\\preprocessing\\final_dataset.csv")
X_train = data_fetcher.get_X_train()
X_test = data_fetcher.get_X_test()
y_train = data_fetcher.get_y_train()
y_test = data_fetcher.get_y_test()



------class run complete------



In [2]:
#NOTE: DON'T REQUIRE SCALING
from sklearn.tree import DecisionTreeClassifier

# Initialize the model
dt_clf = DecisionTreeClassifier(random_state=0)

# Fit the model on the training data
dt_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = dt_clf.predict(X_test)

# Evaluate the model performance
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, y_pred))

#import classification_report
from sklearn.metrics import classification_report
cr1 = classification_report(y_test, y_pred)
print(cr1)

Accuracy: 0.8
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        24
           1       0.82      0.86      0.84        36

    accuracy                           0.80        60
   macro avg       0.79      0.78      0.79        60
weighted avg       0.80      0.80      0.80        60

